<a href="https://colab.research.google.com/github/lukerbs/gsi-internship/blob/master/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import modules
from IPython.display import display, clear_output
import numpy as np
import os

print('S I G N A L   C L A S S I F I E R') 

S I G N A L   C L A S S I F I E R


<h1>Identify Classes of Signals</h1>

In [0]:
# import classes from .txt file 
classes = ['32PSK',
 '16APSK',
 '32QAM',
 'FM',
 'GMSK',
 '32APSK',
 'OQPSK',
 '8ASK',
 'BPSK',
 '8PSK',
 'AM-SSB-SC',
 '4ASK',
 '16PSK',
 '64APSK',
 '128QAM',
 '128APSK',
 'AM-DSB-SC',
 'AM-SSB-WC',
 '64QAM',
 'QPSK',
 '256QAM',
 'AM-DSB-WC',
 'OOK',
 '16QAM']

<h1>Import Signal Data</h1>

In [0]:
path = 'data/npy_data/signal_dataset/'

# load training data
print('Loading training data ...')
x_train = np.load(path + 'train/signals.npy')
y_train = np.load(path + 'train/labels.npy')
snr_train = np.load(path + 'train/snrs.npy')
print('Load complete!')
print('\n')

# load validation data
print('Loading validation data ...')
x_val = np.load(path + 'validation/signals.npy')
y_val = np.load(path + 'validation/labels.npy')
snr_val = np.load(path + 'validation/snrs.npy')
print('Load complete!')
print('\n')

# load testing data
print('Loading testing data ...')
x_test = np.load(path + 'test/signals.npy')
y_test = np.load(path + 'test/labels.npy')
snr_test = np.load(path + 'test/snrs.npy')
print('Load complete!')
print('\n')

Loading training data ...
Load complete!


Loading validation data ...
Load complete!


Loading testing data ...
Load complete!




<h1>Import Deep Learning Libraries</h1>

In [0]:
# import deep learning libraries
import os
import keras
from keras import layers
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D, MaxPooling1D

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

<h1>Create Residual Stack</h1>

In [0]:
# 1d conv resnet
def residual_stack(x, f):
    # 1x1 conv linear
    x = Conv1D(f, 1, strides=1, padding='same', data_format='channels_last')(x)
    x = Activation('linear')(x)
    
    # residual unit 1    
    x_shortcut = x
    x = Conv1D(f, 3, strides=1, padding="same", data_format='channels_last')(x)
    x = Activation('relu')(x)
    x = Conv1D(f, 3, strides=1, padding="same", data_format='channels_last')(x)
    x = Activation('linear')(x)
    # add skip connection
    if x.shape[1:] == x_shortcut.shape[1:]:
      x = Add()([x, x_shortcut])
    else:
      raise Exception('Skip Connection Failure!')
      
    # residual unit 2    
    x_shortcut = x
    x = Conv1D(f, 3, strides=1, padding="same", data_format='channels_last')(x)
    x = Activation('relu')(x)
    x = Conv1D(f, 3, strides = 1, padding = "same", data_format='channels_last')(x)
    x = Activation('linear')(x)
    # add skip connection
    if x.shape[1:] == x_shortcut.shape[1:]:
      x = Add()([x, x_shortcut])
    else:
      raise Exception('Skip Connection Failure!')
      
    # max pooling layer
    x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)
    return x

<h1>Define ResNet Model</h1>

In [0]:
# define resnet model
def ResNet(input_shape, classes):   
    # create input tensor
    x_input = Input(input_shape)
    x = x_input
    # residual stack
    num_filters = 40
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    
    # output layer
    x = Flatten()(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(classes , activation='softmax', kernel_initializer = glorot_uniform(seed=0))(x)
    
    # Create model
    model = Model(inputs = x_input, outputs = x)

    return model

<h1>Option to Save Model Weights and History</h1>

In [0]:
# option to save model weights and model history
save_model = False
save_history = False

# create directory for model weights
if save_model is True:
    weights_path = input("Name model weights directory: ")
    weights_path = "data/weights/" + weights_path

    try:
        os.mkdir(weights_path)
    except OSError:
        print ("Creation of the directory %s failed" % weights_path)
    else:
        print ("Successfully created the directory %s " % weights_path)
    print('\n')
    

# create directory for model history
if save_history is True:
    history_path = input("Name model history directory: ")
    history_path = "data/model_history/" + history_path

    try:
        os.mkdir(history_path)
    except OSError:
        print ("Creation of the directory %s failed" % history_path)
    else:
        print ("Successfully created the directory %s " % history_path)
    print('\n')

<h1>Set Model Parameters</h1>

In [0]:
# reshape input data
x_train = x_train.reshape([-1, 1024, 2])
x_val = x_val.reshape([-1, 1024, 2])
x_test = x_test.reshape([-1, 1024, 2])

# initialize optimizer 
adm = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# set number of epochs
num_epochs = input('Enter number of epochs: ')
num_epochs = int(num_epochs)

# set batch size
batch = 32

# configure weights save

if save_model is True:
    filepath= weights_path + "/{epoch}.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode="auto")
    callbacks_list = [checkpoint]
else:
    callbacks_list = []

Enter number of epochs: 10


<h1>Print Model Summary and Train Network</h1>

In [0]:
# initialize and train model
model = ResNet((1024, 2), 24)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(x_train, y_train, epochs = num_epochs, batch_size = batch, callbacks=callbacks_list, validation_data=(x_val, y_val))

W0903 22:04:53.832963 140633456527104 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 22:04:53.835044 140633456527104 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 22:04:53.839077 140633456527104 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 22:04:53.931131 140633456527104 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0903 22:04:54.402004 140633456527104 deprecation_wrapp

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024, 2)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1024, 40)     120         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 1024, 40)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 1024, 40)     4840        activation_1[0][0]               
__________________________________________________________________________________________________
activation

<h1>Save Model History</h1>

In [0]:
# record model history
train_acc = history.history['acc']
train_loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

if save_history is True:
    # save model history: loss and accuracy
    np.save(history_path + '/train_acc.npy', train_acc)
    np.save(history_path + '/train_loss.npy', train_loss)
    np.save(history_path + '/val_acc.npy', val_acc)
    np.save(history_path + '/val_loss.npy', val_loss)
    print("Model History Saved!")
    print('\n')

<h1>Evaluate Model on Test Data</h1>

In [0]:
# evaluate model on test data
loss, acc = model.evaluate(x_test, y_test, batch_size=32)
print('EVALUATING MODEL ON TEST DATA:')
print('Test Accuracy: ', str(round(acc*100, 2)), '%')
print('\n')